In [14]:
import pandas as pd
import os

In [15]:
#Extract PLUTO Information for each Borough
# -- Brooklyn
brooklynPlutoPath = os.path.join('../../shapeData/externalData/nyc_pluto_14v2/','BK.csv')
brooklynPLUTO = pd.read_csv(brooklynPlutoPath, low_memory=False)
brooklynPLUTO[['BBL']] = brooklynPLUTO[['BBL']].astype(str)
brooklynPLUTO = brooklynPLUTO[['Borough','Block','Lot','CT2010', 'CB2010','ZipCode','BBL', 'Tract2010']]

In [16]:
# -- Bronx
bronxPlutoPath = os.path.join('../../shapeData/externalData/nyc_pluto_14v2/','BX.csv')
bronxPLUTO = pd.read_csv(bronxPlutoPath, low_memory=False)
bronxPLUTO[['BBL']] = bronxPLUTO[['BBL']].astype(str)
bronxPLUTO = bronxPLUTO[['Borough','Block','Lot','CT2010', 'CB2010','ZipCode','BBL', 'Tract2010']]

In [17]:
# -- Manhattan
manhattanPlutoPath = os.path.join('../../shapeData/externalData/nyc_pluto_14v2/','MN.csv')
manhattanPLUTO = pd.read_csv(manhattanPlutoPath, low_memory=False)
manhattanPLUTO[['BBL']] = manhattanPLUTO[['BBL']].astype(str)
manhattanPLUTO = manhattanPLUTO[['Borough','Block','Lot','CT2010', 'CB2010','ZipCode','BBL', 'Tract2010']]

In [18]:
# -- Queens
queensPlutoPath = os.path.join('../../shapeData/externalData/nyc_pluto_14v2/','QN.csv')
queensPLUTO = pd.read_csv(queensPlutoPath, low_memory=False)
queensPLUTO[['BBL']] = queensPLUTO[['BBL']].astype(str)
queensPLUTO = queensPLUTO[['Borough','Block','Lot','CT2010', 'CB2010','ZipCode','BBL', 'Tract2010']]

In [19]:
# -- Staten Island
statenPlutoPath = os.path.join('../../shapeData/externalData/nyc_pluto_14v2/','SI.csv')
statenPLUTO = pd.read_csv(statenPlutoPath, low_memory=False)
statenPLUTO[['BBL']] = statenPLUTO[['BBL']].astype(str)
statenPLUTO = statenPLUTO[['Borough','Block','Lot','CT2010', 'CB2010','ZipCode','BBL', 'Tract2010']]

In [20]:
# Create a dataframe for all Boroughs
nycPluto = pd.concat([brooklynPLUTO,bronxPLUTO,manhattanPLUTO,queensPLUTO,statenPLUTO])

In [21]:
# Etract Finance Data for each Borough
# -- Brooklyn
brooklynFinancePath = os.path.join('../../marketValueData','Brooklyn.csv')
brooklynFinance = pd.read_csv(brooklynFinancePath, low_memory=False)
brooklynFinance.BBLE = brooklynFinance.BBLE.apply(lambda x: x[:10])
brooklynFinance = brooklynFinance[['BBLE','Market_Value']]
brooklynFinance.columns = ['BBLE','Market_Val']

In [22]:
# -- Bronx
bronxFinancePath = os.path.join('../../marketValueData','Bronx.csv')
bronxFinance = pd.read_csv(bronxFinancePath, low_memory=False)
bronxFinance.BBLE = bronxFinance.BBLE.apply(lambda x: x[:10])
bronxFinance = bronxFinance[['BBLE','Market_Val']]

In [23]:
# -- Manattan
manhattanFinancePath = os.path.join('../../marketValueData','Manhattan.csv')
manhattanFinance = pd.read_csv(manhattanFinancePath, low_memory=False)
manhattanFinance.BBLE = manhattanFinance.BBLE.apply(lambda x: x[:10])
manhattanFinance = manhattanFinance[['BBLE','Market_Val']]

In [24]:
# -- Queens
queensFinancePath = os.path.join('../../marketValueData','Queens.csv')
queensFinance = pd.read_csv(queensFinancePath, low_memory=False)
queensFinance.BBLE = queensFinance.BBLE.apply(lambda x: x[:10])
queensFinance = queensFinance[['BBLE','Market_Val']]

In [25]:
# -- Staten Island
statenFinancePath = os.path.join('../../marketValueData','Staten.csv')
statenFinance = pd.read_csv(statenFinancePath, low_memory=False)
statenFinance.BBLE = statenFinance.BBLE.apply(lambda x: x[:10])
statenFinance = statenFinance[['BBLE','Market_Val']]

In [154]:
# Create a dataframe for all Boroughs
nycFinance = pd.concat([brooklynFinance,bronxFinance,manhattanFinance,queensFinance,statenFinance])

In [155]:
# Groupby BBL since those are BBLE records; we took the sum of the marke value for BBLEs that share the same parent BBL
byBBLE = nycFinance.groupby('BBLE')
nycFinanceFinal = byBBLE.sum().reset_index()[['BBLE','Market_Val']]

In [156]:
# Rename the BBLE after grouping to BBL
nycFinanceFinal.rename(columns={'BBLE':'BBL'}, inplace=True)

In [157]:
# Create the master DataFrame for median market value
marketValueBBL = pd.merge(nycPluto,nycFinanceFinal, on='BBL', how='left')

In [158]:
# Create The Zipcode Dataframe 
ByZipCode = marketValueBBL.groupby('ZipCode')
medianMarketValueZipCode = ByZipCode.median().reset_index()[['ZipCode','Market_Val']]

In [159]:
# Remocve the decimal point from CT2010 (else add zeros if it is 1 or 2 digit number)
marketValueBBL['CT2010'] = marketValueBBL.CT2010*100

# Add leading zeros to the CT2010
marketValueBBL['CT2010'] = marketValueBBL['CT2010'].apply(lambda x: '{0:0>8}'.format(x))

# Remove the decimal point
def trim_fraction(text):
    if '.0' in text:
        return text[:text.rfind('.0')]
    return text
marketValueBBL.CT2010 = marketValueBBL.CT2010.apply(trim_fraction)

In [160]:
# change letter to values to create mapping code
marketValueBBL.loc[marketValueBBL['Borough'] == 'BX', ['Borough']] = '005'
marketValueBBL.loc[marketValueBBL['Borough'] == 'BK', ['Borough']] = '047'
marketValueBBL.loc[marketValueBBL['Borough'] == 'MN', ['Borough']] = '061'
marketValueBBL.loc[marketValueBBL['Borough'] == 'QN', ['Borough']] = '081'
marketValueBBL.loc[marketValueBBL['Borough'] == 'SI', ['Borough']] = '085'

In [161]:
# Insert the state index for the mapping code
marketValueBBL['StateFIP'] = '36'
marketValueBBL['CensusTract'] = marketValueBBL['StateFIP'].astype(object) + marketValueBBL['Borough'].astype(object) + marketValueBBL['CT2010'].astype(object)

In [162]:
# Create the census tract dataframe
ByCensusTract = marketValueBBL.groupby('CensusTract')
medianMarketValueCensusTract = ByCensusTract.median().reset_index()[['CensusTract','Market_Val']]

In [163]:
#Write to csv
medianMarketValueZipCode.to_csv('../../shapeData/Tables/medianMarketValueZipCode.csv', sep=',')
medianMarketValueCensusTract.to_csv('../../shapeData/Tables/medianMarketValueCensusTract.csv', sep=',')